In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Set environment variables
import os

os.environ["SCAL_TYPE"] = "complex"
os.environ["PRECISION"] = "double"
os.environ["MY_NUMBA_TARGET"] = "cuda"
 
# Add cle_fun to PYTHON_PATH
import sys
sys.path.append("../../clonscal")

In [2]:
import numba, math # type: ignore 
from numba import cuda # type: ignore 
import numpy as np
from numba import float32
from src.numba_target import myjit

lattice_size = 2**20
threadsperblock = 256
blockspergrid = math.ceil(lattice_size / threadsperblock)

phi0 = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
phi1 = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
eta = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
dS = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
dS_norm = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
mass_real = np.float64(1)
interaction = np.float64(1)
dt_ada = np.float64(0.1)

Using CUDA


In [1]:
import numba, math # type: ignore 
from numba import cuda # type: ignore 
import numpy as np
from numba import float32

lattice_size = 2**20
threadsperblock = 256
blockspergrid = math.ceil(lattice_size / threadsperblock)

phi0 = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
phi1 = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
eta = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
dS = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
dS_norm = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
mass_real = np.float64(1)
interaction = np.float64(1)
dt_ada = np.float64(0.1)

@numba.jit(nogil=True, fastmath=True)
def evolve_kernel(idx, phi0, phi1, dS, eta, dt_ada):
    dt_sqrt = math.sqrt(dt_ada)
    etaterm = eta[idx] * dt_sqrt
    update = etaterm - dt_ada* dS[idx]
    phi1[idx] = phi0[idx] + update

@cuda.jit
def evolve_kernel_compiled(iter_max, phi0, phi1, dS, eta, dt_ada):
    xi = cuda.grid(1)
    if xi < iter_max:
        evolve_kernel(xi, phi0, phi1, dS, eta, dt_ada)

evolve_kernel_compiled[blockspergrid, threadsperblock](lattice_size, phi0, phi1, dS, eta, dt_ada)
cuda.synchronize()
evolve_kernel_compiled[blockspergrid, threadsperblock](lattice_size, phi0, phi1, dS, eta, dt_ada)
cuda.synchronize()
evolve_kernel_compiled[blockspergrid, threadsperblock](lattice_size, phi0, phi1, dS, eta, dt_ada)
cuda.synchronize()
evolve_kernel_compiled[blockspergrid, threadsperblock](lattice_size, phi0, phi1, dS, eta, dt_ada)
cuda.synchronize()
evolve_kernel_compiled[blockspergrid, threadsperblock](lattice_size, phi0, phi1, dS, eta, dt_ada)
cuda.synchronize()
